In [415]:
class OracleState:
    idx = 0
    min_word = ''
    transitions = {}
    supply_fn = 0
    
    def __init__(self, idx=0):
        self.idx = idx
        self.transitions = {}
        self.supply_fn = -1
        
    def add_transition(self, by_letter, to_idx):
        self.transitions[by_letter] = to_idx
        
    def transition_by(self, by_letter):
        return self.transitions[by_letter]

class FactorOracle:
    inp_str = ''
    oracle_sz = 0
    states = []
    
    def __init__(self, inp_str):
        self.reset()
        
        self.inp_str = " " + inp_str
        self.oracle_sz = len(self.inp_str)
        self.__online_init()
        
    def reset(self):
        self.inp_str = ''
        self.oracle_sz = 0
        self.states = []
        
    def __online_init(self):
        self.states.append(OracleState())
        
        for i in range(1, self.oracle_sz):
            self.__add_letter(self.inp_str[i])

    def __add_letter(self, new_letter):
        new_last_idx = len(self.states)
        self.states.append(OracleState(idx=new_last_idx)) # create state m + 1
        self.states[-2].add_transition(to_idx=new_last_idx, by_letter=new_letter) # transition from state m to m + 1
        
        cur_idx = self.states[-2].supply_fn # supply_fn(m)
        while cur_idx > -1 and (new_letter not in self.states[cur_idx].transitions):
            self.states[cur_idx].add_transition(to_idx=new_last_idx, by_letter=new_letter)
            cur_idx = self.states[cur_idx].supply_fn
            
        if cur_idx == -1:
            next_idx = 0
        else:
            next_idx = self.states[cur_idx].transition_by(new_letter)
        self.states[-1].supply_fn = next_idx
        
    def __str__(self):
        out = "Factor Oracle of \"{0}\":\n".format(self.inp_str[1:])
        for i in range(self.oracle_sz):
            st = self.states[i]
            out += "State " +  str(i) \
                + " Supply function " + str(st.supply_fn) \
                + " Transitions " + str(st.transitions) + '\n'
        return out
    
    '''
    Running the deterministic automaton and getting the output
    '''
    def traverse_oracle(self, query_str):
        cur_idx = 0 # starting with state 0
        for char in query_str:   #### HANDLE NOT FOUND CASE ####
            cur_idx = self.state[cur_idx].transitions[char]
        return cur_idx
    
    '''
    Function repet(i), i is a state in Oracle(int_str),
    returns the longest suffix of prefix(i) in inp_str that appears at least twice
    '''
    def suff_repet(self, orc_idx):
        prefix = self.inp_str[:orc_idx]
        pass
        
    '''
    Function poccur(u), u ∈ Fact(int_str),
    returns ending position of first occurrence of factor u in int_str
    '''    
    def first_occur(self, factor): # poccur(u), u ∈ Fact(p)
        begin_pos = self.inp_str.find(factor) #### FIX TO WORK IN O(1) THROUGH AUTOMATON ####
        if begin_pos == - 1:
            pass
        else:
            end_pos = begin_pos + len(factor) - 1
            return end_pos 

In [441]:
a = FactorOracle("abbbaab")
print(a)

b = FactorOracle("abbcabc")
print(b)

c = FactorOracle("abcjiowefcamf")
print(c)

Factor Oracle of "abbbaab":
State 0 Supply function -1 Transitions {'a': 1, 'b': 2}
State 1 Supply function 0 Transitions {'b': 2, 'a': 6}
State 2 Supply function 0 Transitions {'b': 3, 'a': 5}
State 3 Supply function 2 Transitions {'b': 4, 'a': 5}
State 4 Supply function 3 Transitions {'a': 5}
State 5 Supply function 1 Transitions {'a': 6}
State 6 Supply function 1 Transitions {'b': 7}
State 7 Supply function 2 Transitions {}

Factor Oracle of "abbcabc":
State 0 Supply function -1 Transitions {'a': 1, 'b': 2, 'c': 4}
State 1 Supply function 0 Transitions {'b': 2}
State 2 Supply function 0 Transitions {'b': 3, 'c': 4}
State 3 Supply function 2 Transitions {'c': 4}
State 4 Supply function 0 Transitions {'a': 5}
State 5 Supply function 1 Transitions {'b': 6}
State 6 Supply function 2 Transitions {'c': 7}
State 7 Supply function 4 Transitions {}

Factor Oracle of "abcjiowefcamf":
State 0 Supply function -1 Transitions {'a': 1, 'b': 2, 'c': 3, 'j': 4, 'i': 5, 'o': 6, 'w': 7, 'e': 8, 'f': 9

In [452]:
def backward_oracle_matching(word, text):
    fo = FactorOracle(word[::-1]) # initializing factor oracle on mirror image of word
    w_len = len(word)
    t_len = len(text)
    matched_pos = []
    
    cur_pos = 0
    while cur_pos <= t_len - w_len:
        cur_idx = 0
        j = w_len
        while len(fo.states) > cur_idx:
            if text[cur_pos + j] in fo.states[cur_idx].transitions:
                cur_idx = fo.states[cur_idx].transition_by(text[cur_pos + j])
            else:
                cur_idx = len(fo.states) # break
            j -= 1
        
        if j == 0:
            matched_pos.append(cur_pos + 1)
            j = 1
        
        cur_pos += 1
        

In [453]:
backward_oracle_matching("abc", "efjabceufabcweifjabc")

IndexError: string index out of range